In [2]:
#----------IMPORTS------------
import json, re
from pprint import pprint
from lxml import etree
import itertools
from pattern.nl import sentiment
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt


#-----UNBOUND VARIABLES-------

#----------CODE---------------
forums = json.load(open('C:\\Users\\sternheimam\\Dropbox\\TNO\\Scriptie\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-35-45 _amazones_forums_export.json'))
topics = json.load(open('C:\\Users\\sternheimam\\Dropbox\\TNO\\Scriptie\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-36-51_amazones_forum_topics_export.json'))
posts = json.load(open('C:\\Users\\sternheimam\\Dropbox\\TNO\\Scriptie\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-39-20_amazones_forum_posts_export.json'))
users = json.load(open('C:\\Users\\sternheimam\\Dropbox\\TNO\\Scriptie\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-39-20_amazones_users_export.json'))
print('files loaded')

files loaded


In [2]:
# this function replaces ugly links and images by simple, short and readable indicators. It also removes redundant markup from the file (bold, italics, etc).
def cleanup(text):
    # remove links
    text=re.sub('<a.*?>(.*?)</a>','\\1 (LINK)',text)
    # remove emoticons
    text=re.sub('<img.*?title="(.*?)".*?/>', '(EMO:\\1)',text) # sommige emoticons worden ook geschreven als ':shock:', of gewoon als ':)'
    # remove images
    text=re.sub('<img.*?/>', '(IMG)',text)
    # remove quotes
    text=re.sub('<div class="bb-quote">((\s|\S)*?)</div>','(QUOTE)',text)
    # clean up the remains
    return(re.sub('</?(b|p|em|u|i|strong|br|span|div|li)(.*?)/?>','',text))

In [3]:
def header(line):
    if 'Name' in line:
        print(line['Name'])
    if 'Title' in line:
        print(line['Title'])
    if 'Post date' in line:
        print(line['Post date'])
    if 'Datum van inzending' in line:
        print(line['Datum van inzending'])

In [ ]:
# this makes the data super readable yayy
count=0
for x in topics:
    header(x)
    print(cleanup(x['Body']))
    print('\n')
    for y in reversed(posts): # need reversed to sort posts correctly
        if y['Forum Topic ID'] == x['Nid']:
            header(y)
            print(cleanup(y['Body']))
            print('\n')
    print('\n------------- Einde topic -------------\n')

    count+=1 # makes output manageable
    if count>30:
        break

In [35]:
# hier wordt AUTHORLIST gemaakt: een lijst van naam, sentiment, objectivity en vraagtekens voor elke post.
def annotated_data(topics,posts):
    count=0
    questionmarks=0
    sentiment_list=[]
    subjectivity_list=[]
    authorlist=[] # this will contain (author name, question marks, sentiment) for every bit of text ('topic' or 'post')

    #----------------------------------

    for x in topics:
        questionmarks=0
        sentiment_list=[]
#        header(x)
        clean_post=cleanup(x['Body'])
#        print(clean_post)

#        for questionmark in re.findall('\?+',clean_post): 
#            questionmarks+=1
        for sentence in sent_tokenize(clean_post):
            if re.search('\?+',sentence):
                questionmarks = 1
            else:
                questionmarks = 0
            #sentiment_list.append(sentiment(sentence)[0])
            #subjectivity_list.append(sentiment(sentence)[1])
#        authorlist.append((x['Author uid'],np.mean(sentiment_list),np.mean(subjectivity_list),questionmarks,clean_post))
            authorlist.append((sentence,x['Author uid'],sentiment(sentence)[0],sentiment(sentence)[1],questionmarks))

#        print('Question marks:', questionmarks)
#        print('Sentiment:', sentiment_list) # the list is printed
#        print('Subjectivity;', subjectivity_list)
#        print('\n')

        for y in reversed(posts): # need reversed to sort posts correctly
            if y['Forum Topic ID'] == x['Nid']:
                questionmarks=0
                sentiment_list=[]
                subjectivity_list=[]
#                header(y)
                clean_post=cleanup(y['Body'])
#                print(clean_post)

#                for questionmark in re.findall('\?+',clean_post):
#                        questionmarks+=1
                for sentence in sent_tokenize(clean_post):
                    if re.search('\?+',sentence):
                        questionmarks = 1
                    else:
                        questionmarks = 0
                    #sentiment_list.append(sentiment(sentence)[0])
                    #subjectivity_list.append(sentiment(sentence)[1])
#                authorlist.append((y['Auteur-uid'],np.mean(sentiment_list),np.mean(subjectivity_list),questionmarks,clean_post))
                    authorlist.append((sentence,y['Auteur-uid'],sentiment(sentence)[0],sentiment(sentence)[1],questionmarks))

#                print('Question marks:', questionmarks)
#                print('Sentiment:', sentiment_list)
#                print('Subjectivity;', subjectivity_list)
#                print('\n')
#
#        print('\n------------- Einde topic -------------\n')

    #----------------------------------    
        count+=1 # makes output manageable
        if count>100:
            break
    return(authorlist)


In [6]:
# I do not know how to make the list of triples into a dictionary containing two lists as value in ONE step,
# so I make two dictionaries and combine those into one..

def make_dictionary(authorlist):
    author_sent = defaultdict(list)
    author_ques = defaultdict(list)
    for name, s_val, q_val in authorlist:
        author_sent[name].append(s_val) 
        author_ques[name].append(q_val)
    print('dictionaries generated')

    all_dict = defaultdict(list)
    for key in author_sent.keys():
        all_dict[key].append((author_sent[key]))
    for key in author_ques.keys():
        all_dict[key].append(author_ques[key])
    print('dictionaries combined')
    return(all_dict)

In [7]:
# Present the data. Here, I make a plot for every user, over time.
# also save to pdf
def plots_by_author(all_dict):
    pp = PdfPages("plots-author-BVN.pdf")
    for name in all_dict.keys():
        if len(all_dict[name][0]) > 15:
            fig, ax1 = plt.subplots()
            ax2 = ax1.twinx()

            ax1.plot(all_dict[name][0], 'g-', alpha=0.5) # green = sentiment
            ax2.plot(all_dict[name][1], 'r-', alpha=0.5) # red = question marks
            ax1.set_ylabel('Sentiment', color='g')
            ax2.set_ylabel('Question Marks', color='r')

            ax1.set_ylim(-1,1)
            ax2.set_ylim(-5,5)
            plt.title(name)

            figur=plt.figure(1)
            plt.show()
            pp.savefig(figur, dpi = 300, transparent = True)
    pp.close()

In [104]:
# Present the data differently: Here I make a plot for every variable.
# I doubt whether it provides a lot of insight, as I expect that every patient needs different amounts of time to get empowered by the forum
# therefore, empowered and disempowered patients probably level each other out.
# In any case I got to experiment with line plots and scatter plots and trend lines.

def scatter_plots_by_variable(all_dict):
    pp = PdfPages("plots-variable-BVN.pdf")
    j=0
    for j in range(len(all_dict.values()[0])):

        plotlist=[]
        lenlist=[]
        listlist=[]
        
        i=0
        while i>=0:
            somelist=[]

            for name in all_dict:
                if len(all_dict[name][j])>i:
                    somelist.append(all_dict[name][j][i])
                    #print(somelist)
                else:
                    #print(len(all_dict[name][j]),'<',i)
                    pass
            if len(somelist) == 0:
                break
            else:
                listlist.append(somelist)
                plotlist.append(np.mean(somelist))
                lenlist.append(len(somelist))
                i+=1

    # plots showing means per post and trend lines  
        plt.close()        
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()

        x = np.array([i for i in range(len(plotlist))])
        y = plotlist

        ax1.scatter(x,y,s=5, color='green',alpha=0.6) # green = sentiment
        ax2.plot(lenlist, 'r-', alpha=0.2) # red = question marks

        #fit = np.polyfit(x-np.mean(x), y,1)
        #ax1.plot(x, fit[0] * x + fit[1], 'b-', alpha=0.8)

        ax1.set_ylabel('Mean', color='g')
        ax2.set_ylabel('Frequency', color='r')
        ax1.set_xlim(0,)

        if j==0:
            ax1.set_ylim(0,1)
            ax2.set_ylim(0,15)
            plt.title('Mean sentiment per post')
        else:
            ax1.set_ylim(0,6)
            ax2.set_ylim(0,15)
            plt.title('Mean nr of question marks per post')
        j+=1
        
    # scatter plot of all measured values
        #n=0
        #for lists in listlist:
        #    for elements in lists:
        #        ax1.scatter(n,elements, color='blue', s=3, alpha=0.4) # adjusting s to s=len(lists) changes the size of the dots respective to the nr of measurements. would be cool to do with means
        #    n+=1
        figur=plt.figure(1)
        plt.show()
        pp.savefig(figur, dpi = 300, transparent = True)
    pp.close()

In [105]:
plt.close()
print('plots closed')

authorlist=annotated_data(topics, posts)
pprint(authorlist)
#all_dict=make_dictionary(authorlist)
#pprint(all_dict.items())
#plots_by_author(all_dict)
#scatter_plots_by_variable(all_dict)

plots closed
[(u'\nEven een hele praktische vraag: weet iemand of je na een bloedtransfusie zelf naar huis mag/kunt rijden?',
  u'4602',
  -0.004999999999999949,
  0.95,
  1),
 (u'\nIk zou het niet doen.', u'5282', 0.0, 0.0, 0),
 (u'Je weet niet hoe je erop gaat reageren.', u'5282', 0.0, 0.0, 0),
 (u'Hebben ze er niks over gezegd in het ziekenhuis?', u'5282', 0.0, 0.0, 1),
 (u'Ik weet niet hoe ver je moet rijden, maar anders is een taxi misschien te overwegen.',
  u'5282',
  0.03333333333333333,
  0.6166666666666667,
  0),
 (u'Kun je vaak ook nog wel vergoed krijgen.', u'5282', 0.0, 0.0, 0),
 (u'\nDank je Kiki,\n\n\neigenlijk is het ook een domme vraag van me.',
  u'4602',
  -0.35,
  0.6,
  0),
 (u'Je hebt natuurlijk gelijk.', u'4602', 0.05125, 0.205, 0),
 (u'Mijn partner wil wel rijden maar die heeft de afgelopen weken al zoveel werktijd moeten opgeven voor me.',
  u'4602',
  0.0,
  0.0,
  0),
 (u'Ze doet het met liefde (natuurlijk!)', u'4602', 0.34375, 0.55, 0),
 (u'maar ik voel me t

 (u'groetjes en succes', u'5163', 0.0, 0.0, 0),
 (u'\nIk ben af en toe topics aan het verbergen.', u'6095', 0.0, 0.0, 0),
 (u'Echt weggooien doe ik nog niet, want dat is zo drastisch en wil ik niet solo doen.',
  u'6095',
  0.05,
  0.65,
  0),
 (u'Ik zal hier melden welke topics ik weggehaald heb, een lijstje aanleggen.',
  u'6095',
  0.0,
  0.0,
  0),
 (u'Tot nog toe dus gedepubliceerd (topics zijn terug te vinden via find content):\n\n\nop 19-11-2017\n\n\nStel jezelf hier voor [2015]\n\n\nTip: Hoe markeer alle forums als gelezen\n\n\n\xa0\n\n\nop 28-11-2017\n\n\nvragen over de site?',
  u'6095',
  0.35,
  0.85,
  1),
 (u'\nIk zie nergens meer de mogelijkheid om een topic te volgen.',
  u'4602',
  0.0,
  0.0,
  0),
 (u'Zie ik iets over het hoofd?', u'4602', 0.0, 0.0, 1),
 (u"\nYo Dopey, de volgknop van een topic vind je aan de onderkant van het openingsbericht van elk topic, naast de knoppen 'bewerken', 'verwijderen' en 'quote'.",
  u'6095',
  0.1,
  0.25,
  0),
 (u'De knop met de ste

  u'18225',
  0.6333333333333333,
  0.9666666666666667,
  0),
 (u'Bedankt Billeke en Berendina!', u'18225', 0.0, 0.0, 0),
 (u'Ja het wachten nekt me meer dan gedacht.', u'18225', 0.0, 0.0, 0),
 (u'Ben dinsdag geweest voor de punctie dus er gaat een dikke week overheen.',
  u'18225',
  -0.03333333333333333,
  0.5333333333333333,
  0),
 (u'Zouden ze eerder bellen als het niet goed is?',
  u'18225',
  -0.1375,
  0.55,
  1),
 (u'Of denk ik dan te optimistisch?', u'18225', 0.6, 0.9, 1),
 (u'\nDat durf ik niet te zeggen.', u'5647', 0.0, 0.0, 0),
 (u'Heb zelf ook een week gewacht op de uitslag.', u'5647', 0.0, 0.0, 0),
 (u'Maar probeer je gedachten een beetje te verzetten, ondanks dat dit ergens in je achterhoofd zit en zoek zoveel mogelijk afleiding.',
  u'5647',
  0.1,
  0.6,
  0),
 (u'Dat doe ik zelf ook altijd en probeer uit te gaan van het positieve.',
  u'5647',
  0.3,
  0.39999999999999997,
  0),
 (u'Sterkte(EMO:heart)', u'5647', 0.0, 0.0, 0),
 (u'\nWat jammer toch weer dat jonge vrouw

  0.0,
  0),
 (u'Ridoux kan je dus een priv\xe9bericht sturen.', u'6095', 0.0, 0.0, 0),
 (u'\nWachten op uitslagen - een ware ramp vind ik.', u'4877', -0.4, 0.3, 0),
 (u'Komt nog bij dus dat je veel pijn hebt en minder mobiel bent (op zijn zachtst gezegd.)',
  u'4877',
  0.0,
  0.0,
  0),
 (u'Ik hoop dat je organen "schoon" zijn en dat ze snel een goed behandelplan voor je hebben.',
  u'4877',
  0.47222222222222227,
  0.7277777777777777,
  0),
 (u'En dat je zo snel mogelijk minder pijn hebt en beter kunt bewegen.',
  u'4877',
  0.1,
  0.6,
  0),
 (u'Knuffel!', u'4877', 0.0, 0.0, 0),
 (u'D.', u'4877', 0.0, 0.0, 0),
 (u'\nVeel sterkte Ann!', u'4152', 0.0, 0.0, 0),
 (u'Misschien een troost te weten dat een ieder hier weet hoe je je voelt.',
  u'4152',
  0.0,
  0.0,
  0),
 (u'dikke knuffel!', u'4152', -0.041666666666666664, 0.5333333333333333, 0),
 (u'!', u'4152', 0.0, 0.0, 0),
 (u'\nBedankt allemaal, lief te weten dat mensen aan je denken!',
  u'18199',
  0.7916666666666666,
  0.966666666

  u'6095',
  -0.6499999999999999,
  0.9,
  0),
 (u'\nDat klinkt wel goed, maar ik wil dus ook niet in beeld.',
  u'1694',
  0.55,
  0.9,
  0),
 (u'En wat heel duidelijk zou zijn geweest, vooral mensen met uitzaaiingen, die zien zo goed uit, de omgeving ziet daar echt helemaal niks aan, maar dan als het gaat verslechteren, ik baal daar zelf enorm van, al het mooie is verdwenen en of ik kans heb dat het nog terugkomt?',
  u'1694',
  0.5740000000000001,
  0.852,
  1),
 (u'Meestal krijg je die tijd niet meer (EMO:crying).', u'1694', 0.0, 0.0, 0),
 (u'Wimpers weg (ik had lange wimpers), wenkbrauwen weg, mijn krullen weg.....en mijn huid, ziet zo oud uit.',
  u'1694',
  -0.05,
  0.4,
  0),
 (u'Ik werd altijd jaren jonger geschat, voel me nu echt een oud ziek wijf, niet leuk als je zo afscheid neemt, van de wereld\xa0(EMO:sad).',
  u'1694',
  -0.25,
  0.7222222222222223,
  0),
 (u'Wens jullie veel succes.', u'1694', 0.0, 0.0, 0),
 (u'\nOh Truus, wat je schrijft raakt me echt, ik heb daar eige

...

 (u'En inderdaad, 5 jaar hormoontherapie.', u'5483', 0.0, 0.0, 0),
 (u'Ik ben hiermee een gelukkig mens....', u'5483', 0.75, 1.0, 0),
 (u'\nBen met tegenzin begonnen aan de tamoxifen (diagnose 2013, tumor 6cm, \xe9\xe9n aangetaste lymfklier, amputatie, bestraling en chemo), vanwege de mogelijke bijwerkingen.',
  u'4602',
  0.1,
  0.6,
  0),
 (u'Heb het toch braaf een jaar geslikt en ben toen alsnog gestopt.',
  u'4602',
  0.2,
  0.9,
  0),
 (u"Het is 'gewoon' het afwegen van de voor- en nadelen.",
  u'4602',
  0.0,
  0.7,
  0),
 (u'Met de tamoxifen kon ik geen volwaardig leven leiden, wat heb ik er aan tien jaar langer te leven als dat leven niet leefbaar is?',
  u'4602',
  -0.16666666666666666,
  0.5333333333333333,
  1),
 (u'Maar ieder moet deze beslissing voor zichzelf nemen en op basis van haar eigen argumenten en inzichten en in overleg met de oncoloog.',
  u'4602',
  0.0,
  0.3333333333333333,
  0),
 (u'Ik zou bijvoorbeeld niemand anders zomaar adviseren om ook maar te stoppen me

  1.0,
  1),
 (u'\nHoi!', u'4920', 0.0, 0.0, 0),
 (u'Ik ben geopereerd in het Spaarne Gasthuis Hoofddorp, zeer blij met de reconstructie &amp; over het aantrekken van vocht is mij niets bekend.',
  u'4920',
  0.47750000000000004,
  0.9,
  0),
 (u'\n@An77 jij bent verder wel blij met het resultaat?',
  u'5407',
  0.275,
  0.525,
  1),
 (u'Jammergenoeg kan ik nog weinig ziekenhuizen in de buurt vinden die ook monobloc protheses plaatsen (ik woon in overijssel).',
  u'5407',
  0.0,
  0.0,
  0),
 (u'Wel een aantal prive klinieken maar omdat ik nog een preventieve ablatio van links moet is dit niet mogelijk.',
  u'5407',
  -0.10833333333333334,
  0.55,
  0),
 (u'\nMijn huidbesparende diep flap operatie aan beide borsten is maar voor de helft gelukt.Na 10 uur narcose en 5 keer proberen is de linkse kant niet gelukt.Dit vanwege te dunne aders.Nu zit ik met een expander aan de linkse kant en een echte borst rechts.',
  u'6076',
  -0.022222222222222216,
  0.4388888888888889,
  0),
 (u'In novemb

  u'2662',
  -0.275,
  0.9,
  0),
 (u'In het begin zitten ze nog los, omdat ze eerst hun plek moeten vinden en daarna pas vast gaan zitten in het weefsel.',
  u'2662',
  0.06666666666666667,
  0.25,
  0),
 (u'Mijn ene prothese is ondersteboven gedraaid en hij is gelijktijdig ook nog richting mijn oksel gaan kantelen.',
  u'2662',
  0.0,
  0.0,
  0),
 (u'Als je\xa0mijn borst van voren bekeek, keek je tegen de zijkant van het implantaat aan.',
  u'2662',
  0.0,
  0.0,
  0),
 (u'Er liep een duidelijke rand van boven naar beneden in het midden van mijn borst.',
  u'2662',
  -0.09999999999999998,
  0.75,
  0),
 (u'Dat gebeurde bij mij\xa0ongeveer een half-driekwartjaar na de operatie.',
  u'2662',
  0.0,
  0.0,
  0),
 (u'Dat is middels een operatie hersteld.', u'2662', 0.0, 0.0, 0),
 (u'Ik heb een nieuwe prothese gekregen.',
  u'2662',
  0.15000000000000002,
  0.45,
  0),
 (u'\nIk ben 13 juli een diep flap operatie ondergaan en ben nu aan het herstellen.',
  u'6076',
  0.2333333333333333,
 

  0.0,
  0),
 (u'(IMG)', u'1981', 0.0, 0.0, 0),
 (u'\nZo ziet het eruit als het is uitgepakt.', u'1981', 0.0, 0.0, 0),
 (u'Er zitten ook nog steriele handschoenen en een vuilniszak bij.',
  u'1981',
  0.0,
  0.0,
  0),
 (u'Die staan niet op de foto.', u'1981', 0.0, 0.0, 0),
 (u'(IMG)', u'1981', 0.0, 0.0, 0),
 (u"\nHelaas verschijnen bij mij geen foto's, Linnie, alleen een vierkantje!",
  u'4911',
  0.0,
  0.0,
  0),
 (u'\nBij mij ook geen foto - ook geen vierkantjes overigens.',
  u'4877',
  0.0,
  0.0,
  0),
 (u'Nee, ik kan ze helaas ook niet zien.', u'3895', 0.0, 0.0, 0),
 (u'Het klinkt erg heftig allemaal vind ik, maar het lijkt me in jouw situatie echt de beste oplossing.',
  u'3895',
  0.5775,
  1.0,
  0),
 (u'Fijn dat je het zelf thuis kunt doen en niet telkens naar het ziekenhuis hoeft!!',
  u'3895',
  -0.3,
  0.8500000000000001,
  0),
 (u'Ik heb maar een klein beetje ervaring met -een klein beetje- vocht in mijn long, maar ik kon dat ook heel erg goed voelen en merken wanneer h

 (u'\n(QUOTE)', u'17745', 0.0, 0.0, 0),
 (u'\n(QUOTE)', u'17745', 0.0, 0.0, 0),
 (u'\n\xa0\n\n\nLieve amazones, ik heb jullie even nodig.',
  u'2245',
  0.44166666666666665,
  0.6583333333333333,
  0),
 (u'Weer helaas.', u'2245', 0.0, 0.0, 0),
 (u'Even voorgeschiedenis: 2008 tumor in linkerborst (ik was 39).',
  u'2245',
  0.0,
  0.0,
  0),
 (u'Borstsparende ok, bestraling, chemo, herceptin en tamoxifen gehad.',
  u'2245',
  0.0,
  0.0,
  0),
 (u'5 jaar later (45 jaar) een nieuwe tumor in dezelfde borst.',
  u'2245',
  0.05000000000000001,
  0.21666666666666667,
  0),
 (u'Toen borsten geamputeerd en geen nabehandeling.', u'2245', 0.0, 0.0, 0),
 (u'Beide keren waren okselklieren schoon.',
  u'2245',
  0.4666666666666666,
  0.5333333333333333,
  0),
 (u'Vanochtend heb ik in dezelfde borst weer een knobbel gevoeld, exact hetzelfde gevoel als beide eerdere tumoren.',
  u'2245',
  0.0,
  0.21000000000000005,
  0),
 (u'Ligt bovenop de siliconen prothese.', u'2245', 0.0, 0.0, 0),
 (u'Kan dit?

In [106]:
#zorg dat unicode strings worden, zodat python niet gaat huilen bij het wegschrijven naar csv.
for i in range(len(authorlist)):
    authorlist[i] = map(lambda x: x.encode('utf-8') if type(x)==unicode else x, authorlist[i])
    

In [107]:
pprint(authorlist)

[['\nEven een hele praktische vraag: weet iemand of je na een bloedtransfusie zelf naar huis mag/kunt rijden?',
  '4602',
  -0.004999999999999949,
  0.95,
  1],
 ['\nIk zou het niet doen.', '5282', 0.0, 0.0, 0],
 ['Je weet niet hoe je erop gaat reageren.', '5282', 0.0, 0.0, 0],
 ['Hebben ze er niks over gezegd in het ziekenhuis?', '5282', 0.0, 0.0, 1],
 ['Ik weet niet hoe ver je moet rijden, maar anders is een taxi misschien te overwegen.',
  '5282',
  0.03333333333333333,
  0.6166666666666667,
  0],
 ['Kun je vaak ook nog wel vergoed krijgen.', '5282', 0.0, 0.0, 0],
 ['\nDank je Kiki,\n\n\neigenlijk is het ook een domme vraag van me.',
  '4602',
  -0.35,
  0.6,
  0],
 ['Je hebt natuurlijk gelijk.', '4602', 0.05125, 0.205, 0],
 ['Mijn partner wil wel rijden maar die heeft de afgelopen weken al zoveel werktijd moeten opgeven voor me.',
  '4602',
  0.0,
  0.0,
  0],
 ['Ze doet het met liefde (natuurlijk!)', '4602', 0.34375, 0.55, 0],
 ['maar ik voel me toch wel bezwaard.', '4602', 0.0, 0

  0.75,
  0],
 ['Let wel op, het filmpje gaat over pathologie en er komen dode menselijke lichaamsdelen in beeld (vrij discreet in beeld gebracht, maar toch, wel goed om van te voren te weten).',
  '6095',
  0.41111111111111115,
  0.7111111111111111,
  0],
 ['\nP.S.', '6095', 0.0, 0.0, 0],
 ['Ik vond dit topic (LINK) uit 2011 over bestralingsschade en de Bowen-techniek.',
  '6095',
  -0.1,
  0.9,
  0],
 ['Weet iemand of dat hetzelfde is als diepe-fascie therapie?',
  '6095',
  0.0,
  0.3,
  1],
 ['Of misschien een voorloper ervan?', '6095', 0.0, 0.0, 1],
 ['\nNa amputatie beide borsten, is de huid flinterdun geworden.',
  '4827',
  -0.2,
  0.6,
  0],
 ['Het litteken ontsteekt en gaat zelfs open.',
  '4827',
  0.19999999999999998,
  0.46666666666666673,
  0],
 ['De behandelend arts stelt voor om er lichaamseigenvet in te spuiten.',
  '4827',
  0.0,
  0.0,
  0],
 ['Dan krijg je een beschermlaag.', '4827', 0.0, 0.0, 0],
 ['Ook kan een borstreconstroctie (diepflap) in dit geval een oplossi

 ['Ja het wachten nekt me meer dan gedacht.', '18225', 0.0, 0.0, 0],
 ['Ben dinsdag geweest voor de punctie dus er gaat een dikke week overheen.',
  '18225',
  -0.03333333333333333,
  0.5333333333333333,
  0],
 ['Zouden ze eerder bellen als het niet goed is?', '18225', -0.1375, 0.55, 1],
 ['Of denk ik dan te optimistisch?', '18225', 0.6, 0.9, 1],
 ['\nDat durf ik niet te zeggen.', '5647', 0.0, 0.0, 0],
 ['Heb zelf ook een week gewacht op de uitslag.', '5647', 0.0, 0.0, 0],
 ['Maar probeer je gedachten een beetje te verzetten, ondanks dat dit ergens in je achterhoofd zit en zoek zoveel mogelijk afleiding.',
  '5647',
  0.1,
  0.6,
  0],
 ['Dat doe ik zelf ook altijd en probeer uit te gaan van het positieve.',
  '5647',
  0.3,
  0.39999999999999997,
  0],
 ['Sterkte(EMO:heart)', '5647', 0.0, 0.0, 0],
 ['\nWat jammer toch weer dat jonge vrouwen niet serieus worden genomen met zulke klachten.',
  '4877',
  -0.18333333333333335,
  0.6333333333333333,
  0],
 ['We zien het vaker langs komen.'

 ['\nSucces vandaag met de scan.', '4877', 0.0, 0.0, 0],
 ['En met wachten tot woensdag op alle uitslagen.', '4877', 0.0, 0.0, 0],
 ['Ik hoop dat er een perfect behandelplan voor jou wordt opgesteld.',
  '4877',
  1.0,
  1.0,
  0],
 ['Wat de kanker goed aanpakt en wat zo weinig mogelijk bijwerking geeft.',
  '4877',
  0.325,
  0.75,
  0],
 ['\nFijn dat dat er goed uit zag.',
  '3895',
  0.47500000000000003,
  0.8500000000000001,
  0],
 ['Lekker vroeg inderdaad in de scanner...\xc2\xa0\nTegenwoordig heb ik alleen maar CT-scans, dan heb je niet echt de kans om lekker te liggen, maar bij MRIs vroeger viel ik ondanks te herrie soms bijna in slaap, hahaha.',
  '3895',
  0.13,
  0.42000000000000004,
  0],
 ['Heerlijk, zo tussen het werk door.', '3895', 0.6, 0.7000000000000001, 0],
 ['\npfff vast een ding wat er goed uitziet, op naar hetvolgende!',
  '1370',
  -0.08749999999999995,
  0.6555555555555556,
  0],
 ['Succes!', '1370', 0.0, 0.0, 0],
 ['\nLieve allemaal, bedankt voor de berichtjes.'

  -0.13333333333333333,
  0.5333333333333333,
  0],
 ['Dan is het zo slierterig.', '4877', 0.0, 0.0, 0],
 ['En dan de goedgemeende adviezen van vitamine D slikken en zo.',
  '4877',
  0.0,
  0.0,
  0],
 ['Die slik ik al maar dat is de oplossing niet.', '4877', 0.0, 0.0, 0],
 ['Het is wat het is en dus dan maar korter haar.', '4877', 0.0, 0.0, 0],
 ['\nik\xc2\xa0 neem aan dat het over verschillende vormen van kanker gaat.',
  '4602',
  0.0,
  0.2,
  0],
 ["Ochtendrituelen met een stoma of geamputeerde been of arm lijken me dramatisch anders dan\xc2\xa0 voor zo'n ingreep.",
  '4602',
  -0.2333333333333333,
  0.8333333333333335,
  0],
 ['Maar de informatie over deze docu is zo summier dat het niet duidelijk is wie er nu het onderwerp is.',
  '4602',
  -0.16249999999999998,
  0.6,
  0],
 ['\nDank voor de vele reacties en het delen van jullie verhalen.',
  '18186',
  0.0,
  0.0,
  0],
 ['Dit helpt voor ons heel goed omdat we, zoals eerder vermeld, nog in de researchfase van het project zitt

...

  '6095',
  0.4,
  0.8,
  0],
 ['Veel sterkte met je behandeling.', '6095', 0.0, 0.0, 0],
 ['We horen graag hoe het gaat.', '6095', 0.0, 0.0, 0],
 ['\nGoed dat je je hebt aangemeld, en ontzettend alert gereageerd, zeg op de signalen van je lichaam.',
  '4911',
  0.375,
  0.95,
  0],
 ['Heel veel sterkte met de rest van de kuren, hopelijk verdraag je ze goed!',
  '4911',
  0.41041666666666665,
  0.7333333333333334,
  0],
 ['Lieve groetjes, Zangeres',
  '4911',
  0.6333333333333333,
  0.9666666666666667,
  0],
 ['\nLienke succes met je verdere behandeling!', '4877', 0.0, 0.1, 0],
 ['\nWelkom hier, Lienke en succes met je behandelingen, hoop dat de kuren optimaal hun werk zullen doen!',
  '4920',
  0.8375,
  0.9,
  0],
 ['\nDe afgelopen weken waren weer bewogen...', '5717', 0.0, 0.0, 0],
 ['In het ziekenhuis vroeg ik laatst mijn medisch dossier op.',
  '5717',
  0.025,
  0.15000000000000002,
  0],
 ['Voor de arbeidsarts moest ik dit namelijk meenemen.', '5717', 0.0, 0.0, 0],
 ['Daar ik al

  0.4,
  0.9,
  0],
 ['\nBij mij is de tweede borst wel verwijderd omdat ik dat graag wilde.',
  '4877',
  -0.07500000000000001,
  0.75,
  0],
 ['Dus zonder dat daar een afwijking in werd gezien of gevoeld en zonder erfelijk gen. Achteraf maar goed ook want bij weefselonderzoek bleek ook daar kanker in te zitten, 2 mm of zo.',
  '4877',
  0.55,
  0.9,
  0],
 ['Ik ging op mijn gevoel af - dat gevoel zei eraf met allebei.',
  '4877',
  0.0,
  0.0,
  0],
 ['En dat gevoel bleek dus goed.', '4877', 0.55, 0.9, 0],
 ['Heb echt geluk gehad anders had ik na een paar jaar opnieuw gemoeten.',
  '4877',
  0.1,
  0.6000000000000001,
  0],
 ['Ik heb in het ziekenhuis twee vrouwen gesproken die ook hadden gevraagd de andere (gezonde) borst ook weg te halen.',
  '4877',
  0.5,
  0.6000000000000001,
  0],
 ['En bij hen was het geweigerd.', '4877', 0.0, 0.0, 0],
 ['Schijnt dat verzekeraars het dan ook niet vergoeden.', '4877', 0.0, 0.0, 0],
 ['\nMet een verwijzing van\xc2\xa0de chirurg hoort het gewoon 

  0.7041666666666667,
  0],
 ['\nToen\xc2\xa0ik in het ziekenhuis lag,\xc2\xa0kreeg ik extra zuurstof\xc2\xa0omdat mijn zuurstofgehalte te laag was.',
  '1981',
  0.08000000000000002,
  0.54,
  0],
 ['Bij opname 84% en later wisselend\xc2\xa0rond de 90-96%.',
  '1981',
  0.0,
  0.1,
  0],
 ['Bij een gezond persoon moet die toch wel 96% of hoger zijn.',
  '1981',
  0.5,
  0.6000000000000001,
  0],
 ['Die extra zuurstof is best lekker, want je krijgt er toch wat meer energie van.',
  '1981',
  0.5075,
  0.8,
  0],
 ['Net een kleine boost of zo.', '1981', 0.17500000000000002, 0.75, 0],
 ['Ik sliep er savonds mee in het ziekenhuis,\xc2\xa0maar mijn neus begon wel zeer te doen van die kleine uiteindetjes die in je neus zitten.',
  '1981',
  -0.35,
  0.775,
  0],
 ['De verpleegkundige gaf er vaseline voor.', '1981', 0.0, 0.0, 0],
 ['Op een gegeven moment kwam er een fysiotherapeute langs omdat ik steeds zo hoog ademde.',
  '1981',
  0.1,
  0.5499999999999999,
  0],
 ['Laag ademen is lastig a

  0.0,
  0.0,
  0],
 ['te maken."', '4911', 0.0, 0.0, 0],
 ['Bovendien staat de facebookgroep al vermeld in de rubriek: nuttige activiteiten voor Amazones.',
  '4911',
  0.6,
  0.9,
  0],
 ['Ze heeft namelijk al eens eerder gevraagd of ze een oproep mocht plaatsen (in tegenstelling tot nu vroeg ze toen dus eerst toestemming).',
  '4911',
  0.05,
  0.25,
  0],
 ['We hebben toen afgesproken dat we de facebookgroep op die plek zouden vermelden.',
  '4911',
  0.0,
  0.0,
  0],
 ['Dat  moet toch genoeg zijn?', '4911', 0.0, 0.0, 1],
 ['Of worden we nu te streng en jaag je deze mensen dan juist weg?',
  '4911',
  0.0,
  0.8,
  1],
 ['Ik hoor graag van jullie!', '4911', 0.0, 0.0, 0],
 ['\nHeet dat wel \xc2\xb4nacontrole`?',
  '5860',
  0.03333333333333333,
  0.5666666666666668,
  1],
 ['De eerste afspraak bij de chirurg na de behandeling.', '5860', 0.0, 0.0, 0],
 ['Eind september was mijn behandeling afgelopen, afgezien van de hormoonpillen die ik vijf jaar moet nemen.',
  '5860',
  0.0,
  0.0

 ['Dat had ik ook in 2013 toen het doek gevallen was.', '4877', 0.0, 0.0, 0],
 ['Succes met de behandeling - ik hoop dat die goed aanslaat en dat je weinig last van bijwerkingen zult hebben.',
  '4877',
  0.55,
  0.9,
  0],
 ['\nAnnabelle, wat een slecht nieuws.', '6095', -0.7, 0.9, 0],
 ['Rotziekte.', '6095', 0.0, 0.0, 0],
 ['Ik ben het met jou en Stoere Leeuw eens, ik vind weten ook altijd beter te hanteren dan onzekerheid, zelfs al is het slecht nieuws.',
  '6095',
  -0.09999999999999998,
  0.7333333333333334,
  0],
 ['Sterkte meid, als het lukt, hou ons op de hoogte.', '6095', 0.0, 0.0, 0],
 ['\nHe wat naar.', '3895', -0.6499999999999999, 0.9, 0],
 ['Zo zie je maar weer hoe goed je je lichaam vaak kent.',
  '3895',
  0.55,
  0.9,
  0],
 ['Heel eel sterkte met alles dat er weer aan komt nu...',
  '3895',
  0.13333333333333333,
  0.5666666666666668,
  0],
 ['\npffff dat zijn heftige dagen geweest zeg....veel sterkte weer langs de bekende weg!',
  '6103',
  0.26875,
  0.85000000000000

In [99]:
# SOMMIGE NEGATIES WORDEN NIET MEEGENOMEN IN SENTIMENT. simpelweg vervangen van bv 'zonder' geeft niet altijd gewenst resultaat
sentence = "en voor zover ik kon beoordelen zonder vervelende 'bijwerkingen'."
alt_sentence = "en voor zover ik kon beoordelen met geen vervelende 'bijwerkingen'." # 'niet' geeft hetzelfde resultaat

dubneg_sentence = "en voor zover ik kon beoordelen was dat niet zonder vervelende bijwerkingen" # mét vervelende bijwerkingen, dus
alt_dubneg_sentence = "en voor zover ik kon beoordelen was dat niet met geen vervelende bijwerkingen" # deze geeft dus verkeerd resultaat. idem met 'niet'
for sent in [sentence,alt_sentence,dubneg_sentence, alt_dubneg_sentence]:
    print(sentiment(sent)[0])

-0.633333333333
0.316666666667
-0.633333333333
0.316666666667


In [112]:
import csv
data=authorlist

with open('ur file.csv','wb') as out:
    csv_out=csv.writer(out)
    csv_out.writerow([u'sentence',u'author ID',u'sentiment',u'subjectivity',u'question marks'])
    for row in data:
        csv_out.writerow(row)
        print(row)

['\nEven een hele praktische vraag: weet iemand of je na een bloedtransfusie zelf naar huis mag/kunt rijden?', '4602', -0.004999999999999949, 0.95, 1]
['\nIk zou het niet doen.', '5282', 0.0, 0.0, 0]
['Je weet niet hoe je erop gaat reageren.', '5282', 0.0, 0.0, 0]
['Hebben ze er niks over gezegd in het ziekenhuis?', '5282', 0.0, 0.0, 1]
['Ik weet niet hoe ver je moet rijden, maar anders is een taxi misschien te overwegen.', '5282', 0.03333333333333333, 0.6166666666666667, 0]
['Kun je vaak ook nog wel vergoed krijgen.', '5282', 0.0, 0.0, 0]
['\nDank je Kiki,\n\n\neigenlijk is het ook een domme vraag van me.', '4602', -0.35, 0.6, 0]
['Je hebt natuurlijk gelijk.', '4602', 0.05125, 0.205, 0]
['Mijn partner wil wel rijden maar die heeft de afgelopen weken al zoveel werktijd moeten opgeven voor me.', '4602', 0.0, 0.0, 0]
['Ze doet het met liefde (natuurlijk!)', '4602', 0.34375, 0.55, 0]
['maar ik voel me toch wel bezwaard.', '4602', 0.0, 0.0, 0]
['\nVijf mooie portretten van vrouwen na (bors

['\nHier links naar een aantal topics over verschillende soorten reconstructie.', '6095', -0.32499999999999996, 0.55, 0]
['Zoeken met de zoekfunctie boven in het scherm kan natuurlijk ook.', '6095', 0.275, 0.55, 0]
['Filmpje van TheBreastcancersite.com (Engels) waarin verschillende soorten reconstructie goed uitgelegd worden:\n\n\nhttp://blog.thebreastcancersite.com/breastreconstructionanimation/?utm_source=social&amp;utm_medium=wearpr&amp;utm_campaign=breastreconstructionanimation&amp;utm_term=20140805#gsz6fve8S5ZAL6mP.01 (LINK)\n\n\n\xc2\xa0\n\nKeuze: welke reconstructie kan ik laten doen??', '6095', 0.15000000000000002, 0.6666666666666666, 1]
['https://de-amazones.nl/nl/forum/vraag-maar-raak-operatie-en-behandeling-vraag-maar-raak/keuze-welke-reconstructie-kan-ik-laten?page=1 (LINK)\nAlgemeen borstreconstructie topic\xc2\xa0 https://de-amazones.nl/nl/forum/vraag-maar-raak-operatie-en-behandeling-vraag-maar-raak/re-borstreconstructie-algemeen-topic (LINK) \nReconstructie uit buik: DI

['Ik vroeg me ook af.', '18114', 0.1, 0.3, 0]
['Ik ben ook heel moe de laatste tijd +-2maanden.', '18114', -0.05, 0.2, 0]
['Kan dat ermee te makenhebben?', '18114', 0.0, 0.0, 1]
['Ik heb soms ook hartkloppingen uit het niet is maar een paar sec en dan ist weg.', '18114', 0.0, 0.0, 0]
['Ik wil niet overdrijven maar mn angstoverheerst.', '18114', 0.0, 0.0, 0]
['\nIk heb hormoongevoelige borstkanker, 100% oestrogeen-gevoelig en 80% progesteron-gevoelig.', '5522', 0.0, 0.0, 0]
['\nSterkte, Samee, dat je maar heel snel mag weten waar je aan toe bent.', '4911', 0.42000000000000004, 0.75, 0]
['Heb hier inmiddels alles al gelezen, maar dat bk geen pijn zou doen klopt echt niet.', '4911', 0.25, 0.55, 0]
['Te jong is ook geen optie meer, helaas.', '4911', 0.1, 0.4, 0]
['Het hoeft niks te zijn, maar duidelijkheid en uitsluiting lijkt me wel op zijn plek, zeker na je melanoom.', '4911', 0.3, 0.5, 0]
['Het zou moeten kunnen hoor, met protheses?', '4911', 0.0, 0.0, 1]
['Dmv echo ofzo?', '4911', 0.0,

['Ik ben ook een paar keer naar een bijeenkomst van Jong Borstkanker geweest, dat was wel fijn alleen omdat ik uitgezaaide borstkanker heb en de meeste anderen niet, viel ik wel een beetje buiten de boot.', '3895', -0.04999999999999997, 0.7000000000000001, 0]
['Maar via zoiets (of AYA?)', '3895', 0.0, 0.0, 1]
['zou je misschien ook nog iemand, hopelijk een beetje bij jou uit de omgeving, kunnen ontmoeten.', '3895', 0.0, 0.0, 0]
['\nHoi Leonoor, ik woon ook in Noord-Holland en had hormoongevoelige tumor.', '4911', 0.0, 0.0, 0]
['Er zijn vast wel dames uit jouw omgeving!', '4911', 0.0, 0.16666666666666666, 0]
['Hoop dat je wat reacties krijgt!', '4911', 0.0, 0.0, 0]
['Lieve groetjes, Zangeres', '4911', 0.6333333333333333, 0.9666666666666667, 0]
['\nHoi Amazones, ik heb een vraag.', '4911', 0.0, 0.0, 0]
['Ik kwam door chemo in chemopauze, toen aan de Tamoxifen.', '4911', 0.0, 0.0, 0]
['Echter begon ik toch weer met menstrueren.', '4911', 0.0, 0.0, 0]
['Op mijn verzoek is er toen hormoonsp

['De verpleegkundige gaf er vaseline voor.', '1981', 0.0, 0.0, 0]
['Op een gegeven moment kwam er een fysiotherapeute langs omdat ik steeds zo hoog ademde.', '1981', 0.1, 0.5499999999999999, 0]
['Laag ademen is lastig als er veel vocht zit bij je\xc2\xa0longen.', '1981', -0.495, 0.715, 0]
['Het voelt een beetje als hyperventilatie, maar dat schijnt normaal te zijn.', '1981', 0.1, 0.6, 0]
['De fysio\xc2\xa0deed een ademoefening met me waarbij ik\xc2\xa0op een stoel moest zitten en langzaam naar mijn\xc2\xa0navel ademde, net alsof je daar een ballon opblaast.', '1981', -0.04999999999999993, 0.875, 0]
['Ik vond het wel fijn om te doen.', '1981', 0.4, 0.8, 0]
['Alleen lastig om twee seconden de adem vast te houden, maar ik moest niet forceren zei ze.', '1981', -0.375, 0.5583333333333333, 0]
['Anders gewoon naar de navel ademen.', '1981', -0.6499999999999999, 0.9, 0]
['Wat ze ook zei was dat ik niet zo vaak de extra zuurstof moest gebruiken om dat je longen er aan gaan wennen en dat is niet

In [62]:
#u'abcdef'==unicode('abcdef')
unicode('\xa0')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xa0 in position 0: ordinal not in range(128)

In [11]:
sentiment('vereist')

(0.4, 0.9)